# 可选实验室：逻辑回归的梯度下降

## 目标

在本实验室中，您将：

- 更新逻辑回归的梯度下降。
- 在熟悉的数据集上探索梯度下降

In [1]:
import copy, math
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import dlc, plot_data, plt_tumor_data, sigmoid, compute_cost_logistic
from plt_quad_logistic import plt_quad_logistic, plt_prob

plt.style.use('./deeplearning.mplstyle')

## 数据集

让我们从决策边界实验室中使用的相同的两个特征数据集开始。

In [2]:
X_train = np.array([[0.5, 1.5], [1, 1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

和以前一样，我们将使用辅助函数来绘制这些数据。标签$y=1$的数据点显示为红色十字，而标签$y=0$的数据点显示为蓝色圆圈。

In [3]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
plot_data(X_train, y_train, ax)
ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## 逻辑梯度下降

<img align="right" src="../../images/C1_W3_Logistic_gradient_descent.png" style=" width:400px; padding: 10px; " >

回想一下梯度下降算法利用梯度计算：

$$\begin{align*}
&\text{重复直到收敛：} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

其中每次迭代对所有$j$对$w_j$执行同时更新，其中

$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}&=\frac{1}{m}\sum\limits_{i=0}^{m-1}(f_{\mathbf{w},b}(\mathbf{x}^{(i)})-y^{(i)})x_{j}^{(i)}\tag{2}\\
\frac{\partial J(\mathbf{w},b)}{\partial b}&=\frac{1}{m}\sum\limits_{i=0}^{m-1}(f_{\mathbf{w},b}(\mathbf{x}^{(i)})-y^{(i)})\tag{3} 
\end{align*}$$

* m是数据集中训练样本的数量
* $f_{\mathbf{w},b}(x^{(i)})$是模型的预测，而$y^{(i)}$是目标
* 对于逻辑回归模型
  $z=\mathbf{w}\cdot\mathbf{x}+b$
  $f_{\mathbf{w},b}(x)=g(z)$
  其中$g(z)$是sigmoid函数：
  $g(z)=\frac{1}{1+e^{-z}}$

### 梯度下降实现

梯度下降算法的实现有两个组成部分：

- 循环执行上面的等式(1)。这就是下面的`gradient_descent`，通常在可选和练习实验室中提供给您。
- 当前梯度的计算，上面的方程（2，3）。这是下面的`compute_gradient_logistic`。您将被要求实施本周的实践实验室。

#### 计算梯度，代码说明

对所有$w_j$和$b$实现上述方程（2）、（3）。

有很多方法可以实现这一点。下面概述的是这样的：

- 初始化变量以累积`dj_dw`和`dj_db`
- 对于每个例子
     - 计算该示例的误差$g(\mathbf{w}\cdot\mathbf{x}^{(i)}+b)-\mathbf{y}^{(i)}$
     - 对于本例中的每个输入值$x_{j}^{(i)}$，
         - 将误差乘以输入$x_{j}^{(i)}$，并添加到`dj_dw`的相应元素。（上面的等式2）
     - 将错误添加到`dj_db`（上面的等式3）
- 将`dj_db`和`dj_dw`除以示例总数（m）
- 注意NumPy`X[i, :]`或`X[i]`中的$\mathbf{x}^{(i)}$和$x_{j}^{(i)}$是`X[i, j]`

In [4]:
def compute_gradient_logistic(X, y, w, b):
    m, n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.
    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i], w) + b)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i, j]
        dj_db = dj_db + err_i
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    return dj_db, dj_dw

使用下面的单元格检查梯度函数的实现。

In [5]:
X_tmp = np.array([[0.5, 1.5], [1, 1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2., 3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}")
print(f"dj_dw: {dj_dw_tmp.tolist()}")

**期望输出**
``` 
dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]
```

#### 梯度下降代码

实现上面等式（1）的代码在下面实现。花点时间找到例程中的函数并将其与上面的方程进行比较。

In [6]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    J_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    for i in range(num_iters):
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        J_history.append(compute_cost_logistic(X, y, w, b))
        if i % math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
    return w, b, J_history

让我们在数据集上运行梯度下降。

In [10]:
w_tmp = np.zeros_like(X_train[0])
b_tmp = 0.
alph = 0.3
iters = 1000000
w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters)
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

 #### 让我们绘制梯度下降的结果：

In [11]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
plt_prob(ax, w_out, b_out)
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')
ax.axis([0, 4, 0, 3.5])
plot_data(X_train, y_train, ax)
x0 = -b_out / w_out[1]
x1 = -b_out / w_out[0]
ax.plot([0, x0], [x1, 0], c=dlc["dlblue"], lw=1)
plt.show()

在上图中：

- 阴影反映概率 y=1（决策边界之前的结果）
- 决策边界是概率 = 0.5 的线

## 另一个数据集

让我们回到单变量数据集。只需两个参数$w$、$b$，就可以使用等高线图绘制成本函数，以更好地了解梯度下降的作用。

In [12]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0, 0, 0, 1, 1, 1])

和以前一样，我们将使用辅助函数来绘制这些数据。标签$y=1$的数据点显示为红色十字，而标签$y=0$的数据点显示为黑色圆圈。

In [13]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
plt_tumor_data(x_train, y_train, ax)
plt.show()

在下图中，尝试：

- 通过单击右上角的等值线图来更改$w$和$b$。
    - 更改可能需要一两秒钟
    - 注意左上角图上成本的变化值。
    - 请注意，成本是通过每个示例的损失累积的（垂直虚线）
- 单击橙色按钮运行梯度下降。
    - 注意成本的稳步下降（等值线和成本图以log(cost)表示）
    - 单击等值线图将重置模型以进行新的运行
- 要重置绘图，请重新运行单元格

In [14]:
w_range = np.array([-1, 7])
b_range = np.array([1, -14])
quad = plt_quad_logistic(x_train, y_train, w_range, b_range)

## 恭喜！

你有：

- 研究了计算逻辑回归梯度的公式和实现
- 利用这些例程
    - 探索单个变量数据集
    - 探索双变量数据集